# day 23

https://adventofcode.com/2018/day/23

In [132]:
import os

import eri.logging as logging

logging.getLogger('matplotlib').setLevel(logging.WARN)
import pandas as pd
import plotly
import plotly.graph_objs as go
import plotly.offline

plotly.offline.init_notebook_mode(connected=True)

In [2]:
FNAME = os.path.join('data', 'day23.txt')

LOGGER = logging.getLogger('day23')
logging.configure()

## part 1

### problem statement:


> Using your torch to search the darkness of the rocky cavern, you finally locate the man's friend: a small reindeer.
> 
> You're not sure how it got so far in this cave. It looks sick - too sick to walk - and too heavy for you to carry all the way back. Sleighs won't be invented for another 1500 years, of course.
> 
> The only option is experimental emergency teleportation.
> 
> You hit the "experimental emergency teleportation" button on the device and push I accept the risk on no fewer than 18 different warning messages. Immediately, the device deploys hundreds of tiny nanobots which fly around the cavern, apparently assembling themselves into a very specific formation. The device lists the X,Y,Z position (pos) for each nanobot as well as its signal radius (r) on its tiny screen (your puzzle input).
> 
> Each nanobot can transmit signals to any integer coordinate which is a distance away from it less than or equal to its signal radius (as measured by Manhattan distance). Coordinates a distance away of less than or equal to a nanobot's signal radius are said to be in range of that nanobot.
> 
> Before you start the teleportation process, you should determine which nanobot is the strongest (that is, which has the largest signal radius) and then, for that nanobot, the total number of nanobots that are in range of it, including itself.
> 
> For example, given the following nanobots:
> 
>     pos=<0,0,0>, r=4
>     pos=<1,0,0>, r=1
>     pos=<4,0,0>, r=3
>     pos=<0,2,0>, r=1
>     pos=<0,5,0>, r=3
>     pos=<0,0,3>, r=1
>     pos=<1,1,1>, r=1
>     pos=<1,1,2>, r=1
>     pos=<1,3,1>, r=1
> 
> The strongest nanobot is the first one (position 0,0,0) because its signal radius, 4 is the largest. Using that nanobot's location and signal radius, the following nanobots are in or out of range:
> 
> + The nanobot at 0,0,0 is distance 0 away, and so it is in range.
> + The nanobot at 1,0,0 is distance 1 away, and so it is in range.
> + The nanobot at 4,0,0 is distance 4 away, and so it is in range.
> + The nanobot at 0,2,0 is distance 2 away, and so it is in range.
> + The nanobot at 0,5,0 is distance 5 away, and so it is not in range.
> + The nanobot at 0,0,3 is distance 3 away, and so it is in range.
> + The nanobot at 1,1,1 is distance 3 away, and so it is in range.
> + The nanobot at 1,1,2 is distance 4 away, and so it is in range.
> + The nanobot at 1,3,1 is distance 5 away, and so it is not in range.
> 
> In this example, in total, 7 nanobots are in range of the nanobot with the largest signal radius.
> 
> Find the nanobot with the largest signal radius. How many nanobots are in range of its signals?

#### loading data

In [3]:
test_data = """pos=<0,0,0>, r=4
pos=<1,0,0>, r=1
pos=<4,0,0>, r=3
pos=<0,2,0>, r=1
pos=<0,5,0>, r=3
pos=<0,0,3>, r=1
pos=<1,1,1>, r=1
pos=<1,1,2>, r=1
pos=<1,3,1>, r=1"""

In [4]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

In [161]:
import re
import pandas as pd

def parse(s):
    return pd.DataFrame([
        [int(_) for _ in re.match('pos=<([-\d]+),([-\d]+),([-\d]+)>, r=(\d+)', line).groups()]
        for line in s.strip().split('\n')
    ], columns=['x', 'y', 'z', 'r']
    )

In [164]:
parse(test_data)[['x', 'y', 'z']] - [1, 2, 3]

,x,y,z
0,-1,-2,-3
1,0,-2,-3
2,3,-2,-3
3,-1,0,-3
4,-1,3,-3
5,-1,-2,0
6,0,-1,-2
7,0,-1,-1
8,0,1,-2


#### function def

In [188]:
def manhattan(p0, xyz):
    return (xyz - p0).abs().sum(axis=1)

def in_bounds(p0, xyz, r):
    return manhattan(p0, xyz) <= r

In [189]:
recs = parse(test_data)
XYZ = ['x', 'y', 'z']
in_bounds([0, 0, 0], recs[XYZ], 4).sum()

7

In [198]:
def q_1(data):
    recs = parse(data)
    
    p0 = recs.loc[recs.r.idxmax()]
    
    return in_bounds(p0.iloc[:3], recs[XYZ], p0.r).sum()

#### tests

In [199]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 7
    LOGGER.setLevel(logging.INFO)

In [200]:
test_q_1()

#### answer

In [201]:
q_1(load_data())

602

## part 2

### problem statement:

> Now, you just need to figure out where to position yourself so that you're actually teleported when the nanobots activate.
> 
> To increase the probability of success, you need to find the coordinate which puts you in range of the largest number of nanobots. If there are multiple, choose one closest to your position (0,0,0, measured by manhattan distance).
> 
> For example, given the following nanobot formation:
> 
>     pos=<10,12,12>, r=2
>     pos=<12,14,12>, r=2
>     pos=<16,12,12>, r=4
>     pos=<14,14,14>, r=6
>     pos=<50,50,50>, r=200
>     pos=<10,10,10>, r=5
> 
> Many coordinates are in range of some of the nanobots in this formation. However, only the coordinate 12,12,12 is in range of the most nanobots: it is in range of the first five, but is not in range of the nanobot at 10,10,10. (All other coordinates are in range of fewer than five nanobots.) This coordinate's distance from 0,0,0 is 36.
> 
> Find the coordinates that are in range of the largest number of nanobots. What is the shortest manhattan distance between any of those points and 0,0,0?

#### function def

In [236]:
import math
import tqdm

def bounds(recs, col):
    dmin = math.floor(recs[col].min())
    dmax = math.ceil(recs[col].max())
    return dmin, dmax

def num_in_range(recs, xbounds=None, ybounds=None, zbounds=None):
    xmin, xmax = xbounds or bounds(recs, 'x')
    ymin, ymax = ybounds or bounds(recs, 'y')
    zmin, zmax = zbounds or bounds(recs, 'z')
    
    xyz = recs[XYZ]
    r = recs.r
    num_in_range = pd.DataFrame([
        {'x': x, 'y': y, 'z': z, 'n': in_bounds([x, y, z], xyz, r).sum()}
        for x in tqdm.tnrange(int(xmin), int(xmax) + 1, leave=False)
        #for y in tqdm.tnrange(ymin, ymax + 1, leave=False)
        for y in range(int(ymin), int(ymax) + 1)
        #for z in tqdm.tnrange(zmin, zmax + 1, leave=False)
        for z in range(int(zmin), int(zmax) + 1)
    ])
    
    return num_in_range

In [220]:
def scale(recs):
    """find an order of magnitude scale for the records"""
    xmin, xmax = bounds(recs, 'x')
    ymin, ymax = bounds(recs, 'y')
    zmin, zmax = bounds(recs, 'z')
    
    xdelta = xmax - xmin
    ydelta = ymax - ymin
    zdelta = zmax - zmin
    
    return min(math.log10(xdelta), math.log10(ydelta), math.log10(zdelta))


def rescale(recs, i):
    s = scale(recs)
    if s <= i:
        raise ValueError('scale too small, investigate')
    
    d = int(s - i)
    return recs / (10 ** d)

In [415]:
def q_2(data):
    recs = parse(data)
    
    xbounds = ybounds = zbounds = None
    
    s = scale(recs)
    for i in tqdm.tnrange(1, math.ceil(s)):
        # get the records at scale i. find the num_in_range min.
        # use that min to create a bounding box for the next generation
        # iterate
        if i < math.floor(s):
            recs_now = rescale(recs, i)
            recs_now.r += 1
        else:
            recs_now = recs.copy()
        
        nir = num_in_range(recs_now, xbounds, ybounds, zbounds)
        nir.loc[:, 'd0'] = manhattan([0, 0, 0], nir[XYZ])
        k_best = nir.sort_values(by=['n', 'd0'], ascending=[False, True]).iloc[0]
        
        # calculate new bounds based off of previous generation's
        # best k
        low = (k_best - 1) * 10
        high = (k_best + 1) * 10

        xbounds = low.x, high.x
        ybounds = low.y, high.y
        zbounds = low.z, high.z
        
        # iterate
        i += 1
    
    return k_best

dev
<div style="border: 2px solid red;">

##### take a few steps

In [240]:
recs = parse(load_data())
xbounds = ybounds = zbounds = None
i = 1

In [241]:
recs_now = rescale(recs, i)
recs_now[:5]

,x,y,z,r
0,8.076523,2.751078,5.207316,8.670865
1,6.720950,3.569146,3.747442,5.256623
2,7.532963,6.536791,3.372817,7.349089
3,3.277527,4.377105,4.718621,5.340952
4,4.249673,3.462178,6.702001,5.627613


In [242]:
nir = num_in_range(recs_now, xbounds, ybounds, zbounds)
len(nir)

47360

In [258]:
nir.loc[:, 'd0'] = manhattan([0, 0, 0], nir[XYZ])
k_best = nir.sort_values(by=['n', 'd0'], ascending=[False, True]).iloc[0]
k_best

n     789
x       4
y       4
z       5
d0     13
Name: 26698, dtype: int64

we already know this is wrong

##### why doesn't this select the correct first value??

the true answer is known:

In [266]:
answer = pd.Series({'x': 26048747, 'y': 46000568, 'z': 38570787, 'r': 110620102})
answer

x     26048747
y     46000568
z     38570787
r    110620102
dtype: int64

redo the first iteration to figure out how we borked

In [289]:
recs = parse(load_data())
xbounds = ybounds = zbounds = None
i = 1

In [290]:
recs_now = rescale(recs, i)
recs_now.head()

,x,y,z,r
0,8.076523,2.751078,5.207316,8.670865
1,6.720950,3.569146,3.747442,5.256623
2,7.532963,6.536791,3.372817,7.349089
3,3.277527,4.377105,4.718621,5.340952
4,4.249673,3.462178,6.702001,5.627613


In [291]:
nir = num_in_range(recs_now, xbounds, ybounds, zbounds)
len(nir)

47360

In [293]:
nir.loc[:, 'd0'] = manhattan([0, 0, 0], nir[XYZ])
k_best = nir.sort_values(by=['n', 'd0'], ascending=[False, True]).iloc[0]
k_best

n     789
x       4
y       4
z       5
d0     13
Name: 26698, dtype: int64

just to see how the true answer gets rescaled in this setting

In [274]:
answer_now = rescale(pd.concat([answer.to_frame().T, recs]), i).iloc[0]
answer_now

x     2.604875
y     4.600057
z     3.857079
r    11.062010
Name: 0, dtype: float64

In [275]:
recs_now.iloc[0]

x    8.076523
y    2.751078
z    5.207316
r    8.670865
Name: 0, dtype: float64

In [279]:
in_bounds(answer_now.iloc[:3], recs_now[XYZ], np.ceil(recs_now.r)).sum()

967

In [280]:
in_bounds([4, 4, 5], recs_now[XYZ], np.ceil(recs_now.r)).sum()

842

In [284]:
in_bounds([2.5, 4.5, 3.5], recs_now[XYZ], np.ceil(recs_now.r)).sum()

708

so clearly our *actual* answer *would* have scored higher. what confuses me considerably here is that the points surrounding that didn't score highest. I was assuming that the `x` coord would have been 2 or 3 (surrounding 2.60...), e.g.

why not?

maybe it's because the radius rescaled doesn't hit all of the elements inside the unit block -- almost definitely the case. see this:

In [292]:
recs_now_2 = recs_now.copy()
recs_now_2.r += 1
nir_2 = num_in_range(recs_now_2, xbounds, ybounds, zbounds)
len(nir)

47360

In [294]:
nir_2.loc[:, 'd0'] = manhattan([0, 0, 0], nir_2[XYZ])
k_best_2 = nir_2.sort_values(by=['n', 'd0'], ascending=[False, True]).iloc[0]
k_best_2

n     967
x       3
y       5
z       4
d0     12
Name: 25550, dtype: int64

a quick visual of the original dataframe, for fun

In [143]:
df = pd.DataFrame([
    {'x': x, 'y': y, 'z': z, 'ct': ct}
    for (x,  y, z), ct in nir.items()
    if ct
    and 1 < x < 7
    and 1 < y < 7
    and 1 < z < 7
])

df.loc[:, 'sz'] = (df.ct - df.ct.min()) / df.ct.max() * 50
df.shape

(125, 5)

In [146]:
data = [go.Scatter3d(
    x=df.x,
    y=df.y,
    z=df.z,
    mode='markers',
    marker={
        'opacity': 0.7,
        'color': df.ct,
        'colorscale': 'Viridis'
    }
)]
plotly.offline.iplot(data)

let's take a few steps with our new idea -- add 1 to the rescaled radii

##### take a few steps

In [358]:
recs = parse(load_data())
xbounds = ybounds = zbounds = None
i = 1

In [359]:
recs_now = rescale(recs, i)
recs_now[:5]

,x,y,z,r
0,8.076523,2.751078,5.207316,8.670865
1,6.720950,3.569146,3.747442,5.256623
2,7.532963,6.536791,3.372817,7.349089
3,3.277527,4.377105,4.718621,5.340952
4,4.249673,3.462178,6.702001,5.627613


In [360]:
recs_now.r += 1
nir = num_in_range(recs_now, xbounds, ybounds, zbounds)
len(nir)

47360

In [361]:
nir.loc[:, 'd0'] = manhattan([0, 0, 0], nir[XYZ])
k_best = nir.sort_values(by=['n', 'd0'], ascending=[False, True]).iloc[0]
k_best

n     967
x       3
y       5
z       4
d0     12
Name: 25550, dtype: int64

so that is at least (for each component) within .5 of the right value. let's update the bounds and cycle again

In [397]:
low = (k_best - 1) * 10
high = (k_best + 1) * 10

In [398]:
xbounds = low.x, high.x
ybounds = low.y, high.y
zbounds = low.z, high.z

In [399]:
i += 1
print(i)

8


In [400]:
try:
    recs_now = rescale(recs, i)
    recs_now.r += 1
except ValueError:
    recs_now = recs.copy()
    
nir = num_in_range(recs_now, xbounds, ybounds, zbounds)
len(nir)

9261

In [401]:
nir.loc[:, 'd0'] = manhattan([0, 0, 0], nir[XYZ])
k_best = nir.sort_values(by=['n', 'd0'], ascending=[False, True]).iloc[0]
k_best

n           967
x      26048746
y      46000568
z      38570787
d0    110620101
Name: 3031, dtype: int64

In [402]:
answer

x     26048747
y     46000568
z     38570787
r    110620102
dtype: int64

redo the previous four steps until convergence

dev
<div style="border: 2px solid red;">

#### tests

In [404]:
test_data = """pos=<10,12,12>, r=2
pos=<12,14,12>, r=2
pos=<16,12,12>, r=4
pos=<14,14,14>, r=6
pos=<50,50,50>, r=200
pos=<10,10,10>, r=5"""

In [405]:
logging.getLogger('matplotlib').setLevel(logging.WARN)
import plotly
import plotly.graph_objs as go
import plotly.offline

plotly.offline.init_notebook_mode(connected=True)

In [406]:
recs = parse(load_data())

In [408]:
data = [go.Scatter3d(
    x=recs.x,
    y=recs.y,
    z=recs.z,
    mode='markers',
    marker={'opacity': 0.7, 'size': 8}
)]
plotly.offline.iplot(data)

In [ ]:
q_2(test_data)

In [417]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data).d0 == 36
    LOGGER.setLevel(logging.INFO)

In [418]:
test_q_2()

#### answer

In [419]:
q_2(load_data())

n           967
x      26048747
y      46000568
z      38570787
d0    110620102
Name: 3472, dtype: int64

In [32]:
import re
import heapq

def distance(x, y):
    return abs(x[0] - y[0]) + abs(x[1] - y[1]) + abs(x[2] - y[2])

def num_in_range(coord):
    return len([x for x in nanobots if distance(x, coord) <= x[3]])

def get_extents(nanobots):
    maximums = [0, 0, 0]
    minimums = [0, 0, 0]
    for bot in nanobots:
        for i in range(0, 3):
            maximums[i] = max(maximums[i], bot[i])
            minimums[i] = min(minimums[i], bot[i])
    return (maximums[0], maximums[1], maximums[2]), (minimums[0], minimums[1], minimums[2])

def get_corners(cube):
    return [
        cube[0], cube[1],
        (cube[1][0], cube[0][1], cube[0][2]), 
        (cube[0][0], cube[1][1], cube[0][2]), 
        (cube[0][0], cube[0][1], cube[1][2]),
        (cube[1][0], cube[1][1], cube[0][2]),
        (cube[0][0], cube[1][1], cube[1][2]),
        (cube[1][0], cube[0][1], cube[1][2])]
    num_inside = 0

def get_cube_containment(cube, nanobots):
    corners = get_corners(cube)
    num_inside = 0
    for bot in nanobots:
        if bot[0] <= max(cube[0][0], cube[1][0]) and \
            bot[0] >= min(cube[0][0], cube[1][0]) and \
            bot[1] <= max(cube[0][1], cube[1][1]) and \
            bot[1] >= min(cube[0][1], cube[1][1]) and \
            bot[2] <= max(cube[0][2], cube[1][2]) and \
            bot[2] >= min(cube[0][2], cube[1][2]):
            # inside
            num_inside += 1
            continue
        for corner in corners:
            if distance(bot, corner) <= bot[3]:
                num_inside += 1
                break
    return num_inside

def get_center(cube):
    return (int((cube[0][0] + cube[1][0]) / 2), int((cube[0][1] + cube[1][1]) / 2), int((cube[0][2] + cube[1][2]) / 2))
    
def split_cube(orig_cube):
    cubes = []
    center = get_center(orig_cube)
    corners = get_corners(orig_cube)
    raw_cubes = [(center, x) for x in corners]
    out_cubes = []
    for cube in raw_cubes:
        out_cubes.append(((max(cube[0][0], cube[1][0]), max(cube[0][1], cube[1][1]), max(cube[0][2], cube[1][2])),
        (min(cube[0][0], cube[1][0]), min(cube[0][1], cube[1][1]), min(cube[0][2], cube[1][2]))))
    return out_cubes

def score_cubes(cubes, nanobots):
    return [((-get_cube_containment(cube, nanobots), area_of_cube(cube)), cube) for cube in cubes]

def area_of_cube(cube):
    return abs(1 + cube[0][0] - cube[1][0]) * abs(1 + cube[0][1] - cube[1][1]) * abs(1 + cube[0][2] - cube[1][2])

surrounds = [(1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1)]
nanobots = []
with open('data/day23.txt', 'r') as fp:
    for line in fp:
        args = list(map(int, re.findall(r'-?\d+', line)))
        nanobots.append((args[0], args[1], args[2], args[3]))

strongest = max(nanobots, key=lambda x: x[3])
weakest = min(nanobots, key=lambda x: x[3])
in_range = [x for x in nanobots if distance(x, strongest) <= strongest[3]]
print("Num Bots: " + str(len(nanobots)))
print("Strongest: " + str(strongest))
print("Weakest: " + str(weakest))
print("Number In Range Of Strongest: " + str(len(in_range)))

extents = get_extents(nanobots)
print("Extents: " + str(extents))

cubes = split_cube(extents)
scores = score_cubes(cubes, nanobots)

queue = []
for score in scores:
    heapq.heappush(queue, score)

best_small = None
best_small_score = 0
while queue:
    score_tuple, cube = heapq.heappop(queue)
    score, area = score_tuple
    area = area_of_cube(cube)
    print("Next Cube: score=" + str(-score) + " sz=" + str(area) + " " + str(cube) + " queue=" + str(len(queue)))
    cubes = split_cube(cube)
    new_scores = score_cubes(cubes, nanobots)
    for new_score in new_scores:
        if new_score[0][1] > 8 and -new_score[0][0] >= best_small_score: #area
            heapq.heappush(queue, new_score)
        elif new_score[0][1] <= 8:
            if -new_score[0][0] > best_small_score:
                best_small = [new_score[1]]
                best_small_score = -new_score[0][0]
                print("new best " + str(new_score))
            elif -new_score[0][0] == best_small_score:
                best_small.append(new_score[1])

best_point = None
best_point_score = 0
print("Best Small Cube: " + str(best_small))
for cube in best_small:
    for x in range(cube[1][0] , cube[0][0] + 1):
        for y in range(cube[1][1], cube[0][1] + 1):
            for z in range(cube[1][2], cube[0][2] + 1):
                point = (x, y, z)
                score = num_in_range(point)
                if score > best_point_score:
                    best_point_score = score
                    best_point = [point]
                elif score == best_point_score:
                    best_point.append(point)
print("Best Points: " + str(best_point) + " score " + str(best_point_score))
print("Min: " + str(min([(distance(point, (0,0,0)), point) for point in best_point], key=lambda x:x[0])))

Num Bots: 1000
Strongest: (65974757, 7960366, 17449033, 99088599)
Weakest: (41296641, 32421375, 59373580, 49630070)
Number In Range Of Strongest: 602
Extents: ((200144301, 171746322, 192340365), (-170854721, -120589179, -154354348))
Next Cube: score=692 sz=4700148354263561176382592 ((200144301, 171746322, 192340365), (14644790, 25578571, 18993008)) queue=7
Next Cube: score=786 sz=587518544282945147047824 ((107394545, 98662446, 105666686), (14644790, 25578571, 18993008)) queue=14
Next Cube: score=925 sz=73439818882682101697760 ((61019667, 62120508, 62329847), (14644790, 25578571, 18993008)) queue=21
Next Cube: score=962 sz=9179977783992241870611 ((37832228, 43849539, 62329847), (14644790, 25578571, 40661427)) queue=28
Next Cube: score=955 sz=9179977360335262712220 ((37832228, 43849539, 40661427), (14644790, 25578571, 18993008)) queue=35
Next Cube: score=957 sz=1147497388248440336200 ((26238509, 43849539, 40661427), (14644790, 34714055, 29827217)) queue=42
Next Cube: score=963 sz=1434372

Next Cube: score=964 sz=1320 ((26048765, 46000564, 38570794), (26048754, 46000555, 38570784)) queue=426
Next Cube: score=964 sz=1440 ((26048754, 46000564, 38570784), (26048743, 46000555, 38570773)) queue=425
Next Cube: score=964 sz=1440 ((26048765, 46000564, 38570805), (26048754, 46000555, 38570794)) queue=424
Next Cube: score=963 sz=12 ((26048745, 46000567, 38570789), (26048744, 46000566, 38570787)) queue=423
Next Cube: score=963 sz=64 ((26048754, 46000564, 38570794), (26048751, 46000561, 38570791)) queue=422
Next Cube: score=963 sz=180 ((26048748, 46000572, 38570794), (26048743, 46000568, 38570789)) queue=421
Next Cube: score=963 sz=210 ((26048743, 46000568, 38570789), (26048737, 46000564, 38570784)) queue=420
Next Cube: score=963 sz=210 ((26048748, 46000568, 38570784), (26048743, 46000564, 38570778)) queue=419
Next Cube: score=963 sz=69660 ((26048820, 46000564, 38570826), (26048776, 46000529, 38570784)) queue=418
Next Cube: score=962 sz=12 ((26048745, 46000568, 38570789), (26048744,

Next Cube: score=914 sz=8694 ((26048754, 46000546, 38570763), (26048732, 46000529, 38570743)) queue=280
Next Cube: score=913 sz=68040 ((26048732, 46000529, 38570784), (26048688, 46000494, 38570743)) queue=279
Next Cube: score=912 sz=143437202471439479880 ((32035368, 39281797, 46078532), (26238509, 34714055, 40661427)) queue=278
Next Cube: score=910 sz=210 ((26048754, 46000572, 38570778), (26048748, 46000568, 38570773)) queue=277
Next Cube: score=906 sz=1188 ((26048754, 46000572, 38570773), (26048743, 46000564, 38570763)) queue=276
Next Cube: score=906 sz=16725429504 ((26051696, 45999589, 38577109), (26048865, 45997358, 38574464)) queue=275
Next Cube: score=906 sz=143437227215387491638 ((37832228, 39281797, 46078532), (32035368, 34714055, 40661427)) queue=274
Next Cube: score=904 sz=1147497388248440336200 ((37832228, 34714055, 51495637), (26238509, 25578571, 40661427)) queue=273
Next Cube: score=903 sz=180 ((26048737, 46000572, 38570794), (26048732, 46000568, 38570789)) queue=272
Next C

Next Cube: score=805 sz=2090678688 ((26048865, 45998473, 38570496), (26047450, 45997358, 38569174)) queue=136
Next Cube: score=804 sz=4377426154714515 ((26057357, 45705184, 38460727), (25876205, 45562442, 38291443)) queue=135
Next Cube: score=804 sz=17929667039839332928 ((23340079, 46133410, 40661427), (20441649, 43849539, 37952874)) queue=134
Next Cube: score=803 sz=9179978258673481456000 ((61019667, 62120508, 40661427), (37832228, 43849539, 18993008)) queue=133
Next Cube: score=801 sz=143437227215387491638 ((26238509, 39281797, 40661427), (20441649, 34714055, 35244322)) queue=132
Next Cube: score=796 sz=1068932891286 ((26057357, 45981746, 38577109), (26046035, 45972825, 38566529)) queue=131
Next Cube: score=796 sz=4377452013037194 ((26057357, 45705184, 38630012), (25876205, 45562442, 38460727)) queue=130
Next Cube: score=791 sz=17929667039839332928 ((26238509, 46133410, 46078532), (23340079, 43849539, 43369979)) queue=129
Next Cube: score=790 sz=17929660420193928096 ((23340079, 46133

Best Small Cube: [((26048746, 46000568, 38570788), (26048745, 46000567, 38570787)), ((26048748, 46000568, 38570787), (26048747, 46000567, 38570786)), ((26048747, 46000568, 38570787), (26048746, 46000567, 38570786)), ((26048748, 46000567, 38570789), (26048747, 46000566, 38570788)), ((26048748, 46000567, 38570788), (26048747, 46000566, 38570787)), ((26048747, 46000567, 38570788), (26048746, 46000566, 38570787)), ((26048748, 46000568, 38570789), (26048747, 46000567, 38570788)), ((26048747, 46000567, 38570788), (26048746, 46000567, 38570787)), ((26048747, 46000568, 38570789), (26048746, 46000567, 38570788)), ((26048748, 46000567, 38570789), (26048747, 46000567, 38570788)), ((26048748, 46000568, 38570788), (26048747, 46000567, 38570787)), ((26048748, 46000567, 38570788), (26048747, 46000567, 38570787)), ((26048747, 46000568, 38570788), (26048746, 46000567, 38570787)), ((26048748, 46000569, 38570787), (26048747, 46000568, 38570786)), ((26048747, 46000569, 38570787), (26048746, 46000568, 3857

fin